In [1]:
import os
os.chdir('/Users/yongwan89/PycharmProjects/fast_campus_Leture /Weds_class_data_prerocessingLeverup_using_Python/dataset')
import dill

- 모델 학습
- 데이터 불러오기 및 크기 파악

In [2]:
import pandas as pd
df = pd.read_csv("아마존_리뷰_이진_분류_데이터_불균형.csv", engine = 'python')
print(df.shape) # (3367, 12533)

(3367, 12533)


- 결측치 비율 파악

In [3]:
def check_missing_value(df):
    # 결측치 파악해주는 함수
    # df : 결측치 파악 여부를 원하는 데이터 프레임 입력
    len_df_without_missing = len(df.dropna()) #df에서 결측된 행들 모두 제거한 길이
    len_df = len(df)
    missing_records_ratio = (len_df - len_df_without_missing) / len_df
    print('총 {}열 중 {}개에 결측이 발생함. 결측 비율 = {}'.format(len_df, len_df-len_df_without_missing, missing_records_ratio))
    

In [4]:
check_missing_value(df)

총 3367열 중 0개에 결측이 발생함. 결측 비율 = 0.0


- 이상치 파악
    - 값이 0 또는 1이 아닌 value.

In [9]:
'''
(X == 1).sum(axis = 1) : 각 열별로 X=1인 컬럼 갯수 카운트
(X == 0).sum(axis = 1) : 각 열별로 X=0인 컬럼 갯수 카운트
그 둘을 각 열별로 더해서 총 컬럼의 갯수와 안맞는 인덱스 추출
'''
X = df.drop('Label', axis = 1) # 클래스 변수를 제거한 데이터프레임을 X에 저장
outlier_indice = ((X == 1).sum(axis = 1) + (X == 0).sum(axis = 1)) != len(X.columns) # 이상치를 포함하는 샘플 확인
print(sum(outlier_indice))

0


- 라벨 빈도수 파악
    - 다수 클래스와 소수 클래스의 비율이 1:3.27이라서 상대부족 아님.
    - 소수 클래스 비율이 788개여서 절대부족 아님.

In [12]:
# 종속 변수 파악
Y = df['Label']
frequency = Y.value_counts() # -1 : 부정 리뷰, 1 :  긍정 리뷰
print(frequency)
imb_ratio = frequency.iloc[0] / frequency.iloc[-1]
print(imb_ratio)

-1    2579
 1     788
Name: Label, dtype: int64
3.2728426395939088


- 재샘플링 데이터 생성
    - 전체 샘플 수가 적으니 Nearmiss로  샘플링으로 한다.

In [24]:
import imblearn.under_sampling as usam
# rus = usam.RandomUnderSampler()  # 랜덤 언더 샘플링
# rus_X, rus_Y = rus.fit_sample(X, Y)

nrus1 = usam.NearMiss(n_neighbors = 3, version = 1) #nearmiss ver1 객체화
nrus2 = usam.NearMiss(n_neighbors = 3, version = 2) #nearmiss ver2 객체화 Q) 여기에 왜 n_neighbors가 들어가지??
nrus1_X, nrus1_Y = nrus1.fit_sample(X, Y)  #nearmiss ver1 언더샘플링
nrus2_X, nrus2_Y = nrus2.fit_sample(X, Y)  #nearmiss ver2 언더샘플링

origin_X, origin_Y = X, Y # 재샘플링 데이터와의 구분을 위해 이름 넣어서 재정의
# 데이터 프레임화
#rus_X = pd.DataFrame(rus_X, columns = X.columns)
nrus1_X = pd.DataFrame(nrus1_X, columns = X.columns)
nrus2_X = pd.DataFrame(nrus2_X, columns = X.columns)

In [32]:
# 그리드 생성
from sklearn.svm import SVC
from sklearn.naive_bayes import BernoulliNB as BNB

# 훈련 모델1 : Bernoulli Naive Bayes classification
basic_NB = BNB()
# 훈련 모델2 : Bernoulli Naive Bayes classification 비용민감모델
cost_sensitive_NB = BNB(class_prior = [0.5, 0.5]) # 소수와 다수 클래스 확률을 0.5로 맞춰줌/ 재샘플링하지 않을 경우에만 사용.
# 훈련 모델3 : support vector classification
basic_SVM = SVC()
# 훈련 모델3 : support vector classification 비용민감모델
cost_sensitive_SVM = SVC(class_weight = {1:imb_ratio, -1:1})

NB_parameter = {}
SVM_parameter = {'C': [2** -5, 2** -3, 2**-1, 2**1, 2 ** 3, 2**5], 'kernel': ['rbf', 'linear']}

machine_parameter_zip = zip([basic_NB, basic_SVM], [NB_parameter, SVM_parameter])
cost_sensitive_machine_parameter_zip = zip([cost_sensitive_NB, cost_sensitive_SVM], [NB_parameter, SVM_parameter])


- 특징 선택 및 그리드 서치

In [30]:
list(range(1000, 1001, 1000))

[1000]

In [34]:
import warnings
warnings.filterwarnings('ignore', 'FutureWarning')

# 최고값 초기화
from sklearn.model_selection import GridSearchCV
best_score = 0
best_feature_set = []
best_model = ''
best_paramter = ''


# 특징 선택과 그리드 서치
from sklearn.feature_selection import *
for selector in [chi2, mutual_info_classif]: #특징 선택으로 카이제곱과 상호정보량 사용.
    for k in range(1000, 1001, 1000): # 특징 선택을 위한 for문 : 1001 -> 5001로 늘리면 더 많은 경우의수 고려가능
        for (X, Y) in zip([origin_X, nrus1_X, nrus2_X], [origin_Y, nrus1_Y, nrus2_Y]): #zip:(X,Y) 형태로 묵기
            fitted_selector = SelectKBest(selector, k).fit(X, Y)
            column_indice = fitted_selector.get_support()
            columns = X.columns[column_indice]
            
            new_X = pd.DataFrame(fitted_selector.transform(X), columns = columns)                        
            if len(Y) == len(origin_Y): #origin_Y, nrus1_Y, nrus2_Y 중 재샘플링 안된 origin_Y골라서 비용민감모델 적용
                if (Y == origin_Y).all(): # Y랑 origin_Y랑 완전 같은지 확인. 그렇다면 비용 민감 모델 적용 필요
                    for (model, parameter) in cost_sensitive_machine_parameter_zip:    
                        grid = GridSearchCV(estimator = model, param_grid = parameter, cv = 5, scoring = 'f1')
                        grid.fit(new_X, Y)
                        if best_score < grid.best_score_:
                            best_score = grid.best_score_
                            best_model = grid.best_estimator_
                            best_parameter = grid.best_params_
                            best_feature_set = columns
            
            else:
                for (model, parameter) in machine_parameter_zip:
                    grid = GridSearchCV(estimator = model, param_grid = parameter, cv = 5, scoring = 'f1')
                    grid.fit(new_X, Y)
                    if best_score < grid.best_score_:
                        best_score = grid.best_score_
                        best_model = grid.best_estimator_
                        best_parameter = grid.best_params_
                        best_feature_set = columns
            print(best_score)

# 모델 저장
import dill
with open('project1_model.pkl', 'wb') as file:
    dill.dump(best_feature_set, file)
    dill.dump(best_model, file)

0


/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this 

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/sklearn/svm/base.py:196: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this 

0.7540838956641606
0.7540838956641606
0.7540838956641606
0.7540838956641606
0.7540838956641606


In [35]:
best_model

SVC(C=2, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

In [48]:
a =list(best_feature_set)

- 저장한 모델 불러와서 test set으로 훈련

In [40]:
X = pd.read_csv("아마존_리뷰_이진_분류_데이터_불균형_새로들어온데이터.csv", engine = 'python')
X

,pleasantly,brilliant,TRUE,typical,much,Not,yet,realistic,error,human,...,Unnamed: 12522,Unnamed: 12523,Unnamed: 12524,Unnamed: 12525,Unnamed: 12526,Unnamed: 12527,Unnamed: 12528,Unnamed: 12529,Unnamed: 12530,Unnamed: 12531
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [58]:
pd.DataFrame(list(X.columns)).tail(25
                                         )

,0
12507,heavyweight
12508,Unnamed: 12508
12509,Unnamed: 12509
12510,Unnamed: 12510
12511,Unnamed: 12511
12512,Unnamed: 12512
12513,Unnamed: 12513
12514,Unnamed: 12514
12515,Unnamed: 12515
12516,Unnamed: 12516


In [56]:
len(X.columns)

12532

In [38]:
## 모델 활용
import dill


with open('project1_model.pkl', 'rb') as file:
    feature_set = dill.load(file)
    model = dill.load(file)

X = X[feature_set]
prediction_result = model.predict(X)
print(prediction_result)

KeyError: "['somehow'] not in index"